<a href="https://colab.research.google.com/github/carvalheirafc/mnist-MachineLearning-Supervised/blob/main/boosting_and_bagging/stacking_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import 

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import BaggingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import StackingClassifier
from sklearn import tree
import time

### Data Input and miscs


In [2]:
data_X = np.load('./drive/My Drive/mnist_datasets/mnist_X.npy')
data_y = np.load('./drive/My Drive/mnist_datasets/mnist_labels.npy')

In [3]:
half_X = np.array(np.split(data_X, 2))
half_y = np.array(np.split(data_y, 2))

print('Mnist 50% Data: \n')
print('X: ', half_X[0].shape)
print('y: ', half_y[0].shape)

Mnist 50% Data: 

X:  (30000, 1, 28, 28)
y:  (30000, 1)


In [4]:
smaller_X = np.array(np.split(data_X, 5))
smaller_y = np.array(np.split(data_y, 5))

print('Mnist 20% Data: \n')
print('X: ', smaller_X[0].shape)
print('y: ', smaller_y[0].shape)

Mnist 20% Data: 

X:  (12000, 1, 28, 28)
y:  (12000, 1)


In [5]:
data_two_classes = np.load('./drive/My Drive/mnist_datasets/two_classes_data.npy', allow_pickle=True)
two_classes_X = np.concatenate((data_two_classes[0][0], data_two_classes[1][0]), axis=0)
two_classes_y = np.concatenate((data_two_classes[0][1], data_two_classes[1][1]), axis=0)
print('Mnist Two Classes: \n')
print('Zeros: ', data_two_classes[0][0].shape)
print('Ones:  ', data_two_classes[1][0].shape)
print('X:     ', two_classes_X.shape)
print('y:     ', two_classes_y.shape)


Mnist Two Classes: 

Zeros:  (5923, 28, 28)
Ones:   (6742, 28, 28)
X:      (12665, 28, 28)
y:      (12665,)


### Hold-Out Sets


In [6]:
ho_two_class_91_train_X, ho_two_class_91_test_X, ho_two_class_91_train_y, ho_two_class_91_test_y = train_test_split(two_classes_X, two_classes_y, train_size=0.9, random_state=555)
ho_two_class_82_train_X, ho_two_class_82_test_X, ho_two_class_82_train_y, ho_two_class_82_test_y = train_test_split(two_classes_X, two_classes_y, train_size=0.8, random_state=555)
ho_two_class_73_train_X, ho_two_class_73_test_X, ho_two_class_73_train_y, ho_two_class_73_test_y = train_test_split(two_classes_X, two_classes_y, train_size=0.7, random_state=555)

print('Hold-Out Mnist Two Classes[0, 1] Split\n')
print('9/1: ', ho_two_class_91_train_X.shape)
print('8/2: ', ho_two_class_82_train_X.shape)
print('7/3: ', ho_two_class_73_train_X.shape)

Hold-Out Mnist Two Classes[0, 1] Split

9/1:  (11398, 28, 28)
8/2:  (10132, 28, 28)
7/3:  (8865, 28, 28)


In [7]:
ho_smaller_91_train_X, ho_smaller_91_test_X, ho_smaller_91_train_y, ho_smaller_91_test_y = train_test_split(smaller_X[0], smaller_y[0], train_size=0.9, random_state=555)
ho_smaller_82_train_X, ho_smaller_82_test_X, ho_smaller_82_train_y, ho_smaller_82_test_y = train_test_split(smaller_X[0], smaller_y[0], train_size=0.8, random_state=555)
ho_smaller_73_train_X, ho_smaller_73_test_X, ho_smaller_73_train_y, ho_smaller_73_test_y = train_test_split(smaller_X[0], smaller_y[0], train_size=0.7, random_state=555)

print('Hold-Out Half Data Split\n')
print('9/1: ', ho_smaller_91_train_X.shape)
print('8/2: ', ho_smaller_82_train_X.shape)
print('7/3: ', ho_smaller_73_train_X.shape)

Hold-Out Half Data Split

9/1:  (10800, 1, 28, 28)
8/2:  (9600, 1, 28, 28)
7/3:  (8400, 1, 28, 28)


### Custom MLP 

In [8]:
class customMLPClassifer(MLPClassifier):
    
    def resample_with_replacement(self, X_train, y_train, sample_weight):

        # normalize sample_weights if not already
        sample_weight = sample_weight / sample_weight.sum(dtype=np.float64)

        X_train_resampled = np.zeros((len(X_train), len(X_train[0])), dtype=np.float32)
        y_train_resampled = np.zeros((len(y_train)), dtype=np.int)
        for i in range(len(X_train)):
            # draw a number from 0 to len(X_train)-1
            draw = np.random.choice(np.arange(len(X_train)), p=sample_weight)

            # place the X and y at the drawn number into the resampled X and y
            X_train_resampled[i] = X_train[draw]
            y_train_resampled[i] = y_train[draw]

        return X_train_resampled, y_train_resampled


    def fit(self, X, y, sample_weight=None):
        if sample_weight is not None:
            X, y = self.resample_with_replacement(X, y, sample_weight)

        return self._fit(X, y, incremental=(self.warm_start and
                                            hasattr(self, "classes_")))

In [9]:
mlp_classifier = customMLPClassifer().set_params(activation='relu', solver='adam')
print(mlp_classifier)

customMLPClassifer(activation='relu', alpha=0.0001, batch_size='auto',
                   beta_1=0.9, beta_2=0.999, early_stopping=False,
                   epsilon=1e-08, hidden_layer_sizes=(100,),
                   learning_rate='constant', learning_rate_init=0.001,
                   max_fun=15000, max_iter=200, momentum=0.9,
                   n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
                   random_state=None, shuffle=True, solver='adam', tol=0.0001,
                   validation_fraction=0.1, verbose=False, warm_start=False)


### Decision Tree


In [10]:
'''
Default ccp_alfa(Fator de confiança)
'''

default_tree_classifier = tree.DecisionTreeClassifier(ccp_alpha=0.0)

### Estimators


In [11]:
estimators = [
              ('mlp', mlp_classifier),
              ('dt', default_tree_classifier)
]

### Stacking with Bagging

In [12]:
'''
Two Classes Data-set
'''


for estimator in [10, 15, 20]:
  clf = StackingClassifier(estimators=estimators, final_estimator=BaggingClassifier(n_estimators=estimator))
  print('Bagging Estimators: ', estimator)
  print('Two Class Data-set | 9/1 Holdout')
  start_time = time.time()
  clf.fit(ho_two_class_91_train_X.reshape(ho_two_class_91_train_X.shape[0], -1), ho_two_class_91_train_y)
  print("Execution Time: %s seconds" % (time.time() - start_time))
  ho_two_class_91_pred_y = clf.predict(ho_two_class_91_test_X.reshape(ho_two_class_91_test_X.shape[0], -1))
  matrix = metrics.confusion_matrix(ho_two_class_91_test_y, ho_two_class_91_pred_y)
  print('Acurácia:', np.trace(matrix) / len(ho_two_class_91_test_y) * 100, '%')
  print('-------------------------------------------------- \n\n')

for estimator in [10, 15, 20]:
  clf = StackingClassifier(estimators=estimators, final_estimator=BaggingClassifier(n_estimators=estimator))
  print('Bagging Estimators: ', estimator)
  print('Two Class Data-set | 8/2 Holdout')
  start_time = time.time()
  clf.fit(ho_two_class_82_train_X.reshape(ho_two_class_82_train_X.shape[0], -1), ho_two_class_82_train_y)
  print("Execution Time: %s seconds" % (time.time() - start_time))
  ho_two_class_82_pred_y = clf.predict(ho_two_class_82_test_X.reshape(ho_two_class_82_test_X.shape[0], -1))
  matrix = metrics.confusion_matrix(ho_two_class_82_test_y, ho_two_class_82_pred_y)
  print('Acurácia:', np.trace(matrix) / len(ho_two_class_82_test_y) * 100, '%')
  print('-------------------------------------------------- \n\n')

for estimator in [10, 15, 20]:
  clf = StackingClassifier(estimators=estimators, final_estimator=BaggingClassifier(n_estimators=estimator))
  print('Bagging Estimators: ', estimator)
  print('Two Class Data-set | 7/3 Holdout')
  start_time = time.time()
  clf.fit(ho_two_class_73_train_X.reshape(ho_two_class_73_train_X.shape[0], -1), ho_two_class_73_train_y)
  print("Execution Time: %s seconds" % (time.time() - start_time))
  ho_two_class_73_pred_y = clf.predict(ho_two_class_73_test_X.reshape(ho_two_class_73_test_X.shape[0], -1))
  matrix = metrics.confusion_matrix(ho_two_class_73_test_y, ho_two_class_73_pred_y)
  print('Acurácia:', np.trace(matrix) / len(ho_two_class_73_test_y) * 100, '%')
  print('-------------------------------------------------- \n\n')

Bagging Estimators:  10
Two Class Data-set | 9/1 Holdout
Execution Time: 53.94653010368347 seconds
Acurácia: 99.92107340173638 %
-------------------------------------------------- 


Bagging Estimators:  15
Two Class Data-set | 9/1 Holdout
Execution Time: 52.52971339225769 seconds
Acurácia: 100.0 %
-------------------------------------------------- 


Bagging Estimators:  20
Two Class Data-set | 9/1 Holdout
Execution Time: 56.66041803359985 seconds
Acurácia: 100.0 %
-------------------------------------------------- 


Bagging Estimators:  10
Two Class Data-set | 8/2 Holdout
Execution Time: 55.04527521133423 seconds
Acurácia: 99.96052112120016 %
-------------------------------------------------- 


Bagging Estimators:  15
Two Class Data-set | 8/2 Holdout
Execution Time: 52.38549852371216 seconds
Acurácia: 100.0 %
-------------------------------------------------- 


Bagging Estimators:  20
Two Class Data-set | 8/2 Holdout
Execution Time: 51.179165840148926 seconds
Acurácia: 99.96052112

In [13]:
'''
Smaller Size and Holdout sets
'''

for estimator in [10, 15, 20]:
  clf = StackingClassifier(estimators=estimators, final_estimator=BaggingClassifier(n_estimators=estimator))
  print('Bagging Estimators: ', estimator)
  print('Smaller Data-set | 9/1 Holdout')
  start_time = time.time()
  clf.fit(ho_smaller_91_train_X.reshape(ho_smaller_91_train_X.shape[0], -1), ho_smaller_91_train_y.ravel())
  print("Execution Time: %s seconds" % (time.time() - start_time))
  ho_smaller_91_pred_y = clf.predict(ho_smaller_91_test_X.reshape(ho_smaller_91_test_X.shape[0], -1))
  matrix = metrics.confusion_matrix(ho_smaller_91_test_y, ho_smaller_91_pred_y)
  print('Acurácia:', np.trace(matrix) / len(ho_smaller_91_test_y) * 100, '%')
  print('-------------------------------------------------- \n\n')

for estimator in [10, 15, 20]:
  clf = StackingClassifier(estimators=estimators, final_estimator=BaggingClassifier(n_estimators=estimator))
  print('Bagging Estimators: ', estimator)
  print('Smaller Data-set | 8/2 Holdout')
  start_time = time.time()
  clf.fit(ho_smaller_82_train_X.reshape(ho_smaller_82_train_X.shape[0], -1), ho_smaller_82_train_y.ravel())
  print("Execution Time: %s seconds" % (time.time() - start_time))
  ho_smaller_82_pred_y = clf.predict(ho_smaller_82_test_X.reshape(ho_smaller_82_test_X.shape[0], -1))
  matrix = metrics.confusion_matrix(ho_smaller_82_test_y, ho_smaller_82_pred_y)
  print('Acurácia:', np.trace(matrix) / len(ho_smaller_82_test_y) * 100, '%')
  print('-------------------------------------------------- \n\n')

for estimator in [10, 15, 20]:
  clf = StackingClassifier(estimators=estimators, final_estimator=BaggingClassifier(n_estimators=estimator))
  print('Bagging Estimators: ', estimator)
  print('Smaller Data-set | 7/3 Holdout')
  start_time = time.time()
  clf.fit(ho_smaller_73_train_X.reshape(ho_smaller_73_train_X.shape[0], -1), ho_smaller_73_train_y.ravel())
  print("Execution Time: %s seconds" % (time.time() - start_time))
  ho_smaller_73_pred_y = clf.predict(ho_smaller_73_test_X.reshape(ho_smaller_73_test_X.shape[0], -1))
  matrix = metrics.confusion_matrix(ho_smaller_73_test_y, ho_smaller_73_pred_y)
  print('Acurácia:', np.trace(matrix) / len(ho_smaller_73_test_y) * 100, '%')
  print('-------------------------------------------------- \n\n')

Bagging Estimators:  10
Smaller Data-set | 9/1 Holdout
Execution Time: 80.472665309906 seconds
Acurácia: 94.08333333333333 %
-------------------------------------------------- 


Bagging Estimators:  15
Smaller Data-set | 9/1 Holdout
Execution Time: 88.91025161743164 seconds
Acurácia: 93.5 %
-------------------------------------------------- 


Bagging Estimators:  20
Smaller Data-set | 9/1 Holdout
Execution Time: 96.36534404754639 seconds
Acurácia: 94.33333333333334 %
-------------------------------------------------- 


Bagging Estimators:  10
Smaller Data-set | 8/2 Holdout
Execution Time: 79.54090762138367 seconds
Acurácia: 94.20833333333334 %
-------------------------------------------------- 


Bagging Estimators:  15
Smaller Data-set | 8/2 Holdout
Execution Time: 83.02668166160583 seconds
Acurácia: 92.41666666666667 %
-------------------------------------------------- 


Bagging Estimators:  20
Smaller Data-set | 8/2 Holdout
Execution Time: 84.73220086097717 seconds
Acurácia: 92.